In [ ]:
import datetime as dt
import sys
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import pandas as pd
from pandas_datareader import data as web
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib.cm as cm
!pip install arch
from arch import arch_model
from numpy.linalg import LinAlgError
from scipy import stats
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot, moment
from arch import arch_model
from arch.univariate import ConstantMean, GARCH, Normal
from sklearn.model_selection import TimeSeriesSplit
import warnings


In [ ]:

%matplotlib inline
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
sns.set(style="darkgrid", color_codes=True)
rcParams['figure.figsize'] = 8,4

In [ ]:
def hurst(ts):
    lags = range(2, 100)
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = polyfit(log(lags), log(tau), 1)
    return poly[0]*2.0

In [ ]:
def plot_correlogram(x, lags=None, title=None):
    lags = min(10, int(len(x)/5)) if lags is None else lags
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
    x.plot(ax=axes[0][0])
    q_p = np.max(q_stat(acf(x, nlags=lags), len(x))[1])
    stats = f'Q-Stat: {np.max(q_p):>8.2f}\nADF: {adfuller(x)[1]:>11.2f} \nHurst: {round(hurst(x.values),2)}'
    axes[0][0].text(x=.02, y=.85, s=stats, transform=axes[0][0].transAxes)
    probplot(x, plot=axes[0][1])
    mean, var, skew, kurtosis = moment(x, moment=[1, 2, 3, 4])
    s = f'Mean: {mean:>12.2f}\nSD: {np.sqrt(var):>16.2f}\nSkew: {skew:12.2f}\nKurtosis:{kurtosis:9.2f}'
    axes[0][1].text(x=.02, y=.75, s=s, transform=axes[0][1].transAxes)
    plot_acf(x=x, lags=lags, zero=False, ax=axes[1][0])
    plot_pacf(x, lags=lags, zero=False, ax=axes[1][1])
    axes[1][0].set_xlabel('Lag')
    axes[1][1].set_xlabel('Lag')
    fig.suptitle(title, fontsize=20)
    fig.tight_layout()
    fig.subplots_adjust(top=.9)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
amzn = pd.read_csv('/content/drive/MyDrive/Quant Method Project - Amazon /Amazon Stock Data 5years.csv')

In [ ]:
start = pd.Timestamp('2024-08-30')
end = pd.Timestamp('2024-12-06')

amzn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        1258 non-null   object
 1   Close/Last  1258 non-null   object
 2   Volume      1258 non-null   int64 
 3   Open        1258 non-null   object
 4   High        1258 non-null   object
 5   Low         1258 non-null   object
dtypes: int64(1), object(5)
memory usage: 59.1+ KB


In [ ]:
amzn.head()

,Date,Close/Last,Volume,Open,High,Low
0,12/06/2024,$227.03,44178070,$220.75,$227.15,$220.60
1,12/05/2024,$220.55,41140220,$218.03,$222.15,$217.30
2,12/04/2024,$218.16,48745720,$215.96,$220.00,$215.75
3,12/03/2024,$213.44,32214830,$210.31,$214.02,$209.65
4,12/02/2024,$210.71,39523190,$209.96,$212.99,$209.5101


In [ ]:
amzn.rename(columns={'Close/Last': 'Close'}, inplace=True)

In [ ]:
amzn['Return'] = 100 * (amzn['Close'].pct_change())
amzn['Log_Return'] = np.log(amzn['Close']).diff().mul(100)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
amzn['Close'] = amzn['Close'].astype(str)
amzn['Close'] = amzn['Close'].str.replace(r'[^0-9.]', '', regex=True).astype(float)

In [ ]:
print(amzn['Close'].unique())

In [ ]:
amzn = amzn.dropna(subset=['Close'])

In [ ]:
amzn['Return'] = 100 * (amzn['Close'].pct_change())
amzn['Log_Return'] = np.log(amzn['Close']).diff().mul(100)
amzn = amzn.dropna()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
max_lags = min(8, len(amzn['Log_Return']) - 1)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
plot_acf(amzn['Log_Return'], lags=max_lags, ax=axes[0])
plot_pacf(amzn['Log_Return'], lags=max_lags, ax=axes[1])
plt.suptitle('Amazon Log Returns: ACF and PACF')
plt.show()


In [ ]:
print(len(amzn['Log_Return'].sub(amzn['Log_Return'].mean()).pow(2)))

In [ ]:
valid_lags = min(8, len(amzn['Log_Return']) - 1)
plot_correlogram(amzn['Log_Return'].sub(amzn['Log_Return'].mean()).pow(2), lags=valid_lags, title='Amazon Daily Volatility')

In [ ]:
std_daily = amzn['Return'].std()
print(f'Daily volatility: {round(std_daily,2)}%')

std_monthly = np.sqrt(21) * std_daily
print(f'\nMonthly volatility: {round(std_monthly,2)}%')

std_quarterly = np.sqrt(4 * 21) * std_daily
print(f'\nQuarterly volatility: {round(std_quarterly,2)}%')

std_annual = np.sqrt(252) * std_daily
print(f'\nAnnual volatility: {round(std_annual,2)}%')

In [ ]:
def simulate_GARCH(n, omega, alpha, beta = 0):
    np.random.seed(4)
    white_noise = np.random.normal(size = n)
    resid = np.zeros_like(white_noise)
    variance = np.zeros_like(white_noise)
    for t in range(1, n):
        variance[t] = omega + alpha * resid[t-1]**2 + beta * variance[t-1]
        resid[t] = np.sqrt(variance[t]) * white_noise[t]

    return resid, variance

In [ ]:
arch_resid, arch_variance = simulate_GARCH(n= 200,
                                           omega = 0.1, alpha = 0.7)
garch_resid, garch_variance = simulate_GARCH(n= 200,
                                             omega = 0.1, alpha = 0.7,
                                             beta = 0.1)
plt.figure(figsize=(10,5))
plt.plot(arch_variance, color = 'red', label = 'ARCH Variance')
plt.plot(garch_variance, color = 'orange', label = 'GARCH Variance')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
sim_resid, sim_variance = simulate_GARCH(n = 200,  omega = 0.1, alpha = 0.3, beta = 0.2)
plt.plot(sim_variance, color = 'orange', label = 'Variance')
plt.plot(sim_resid, color = 'green', label = 'Residuals')
plt.title('First simulated GARCH, Beta = 0.2')
plt.legend(loc='best')
plt.show()
plt.figure(figsize=(10,3))
sim_resid, sim_variance = simulate_GARCH(n = 200,  omega = 0.1, alpha = 0.3, beta = 0.6)
plt.plot(sim_variance, color = 'red', label = 'Variance')
plt.plot(sim_resid, color = 'deepskyblue', label = 'Residuals')
plt.title('Second simulated GARCH, Beta = 0.6')
plt.legend(loc='best')
plt.show()

In [ ]:
basic_gm = arch_model(amzn['Return'], p = 1, q = 1,
                      mean = 'constant', vol = 'GARCH', dist = 'normal')
gm_result = basic_gm.fit(update_freq = 4)

In [ ]:
print(gm_result.summary())

In [ ]:
gm_result.plot()
plt.show()

In [ ]:
gm_forecast = gm_result.forecast(horizon = 5)
print(gm_forecast.variance[-1:])

In [ ]:
gm_resid = gm_result.resid
gm_std = gm_result.conditional_volatility
gm_std_resid = gm_resid /gm_std
plt.figure(figsize=(7,4))
sns.distplot(gm_std_resid, norm_hist=True, fit=stats.norm, bins=50, color='r')
plt.legend(('normal', 'standardized residuals'))
plt.show()

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Quant Method Project - Amazon /amzn-options-clean.csv')

In [ ]:
df.head(10)

,Strike,Moneyness,Mid,Last,Delta,IV,Type,Time
0,80.0,64.63%,147.10,122.48,0.99459,258.94%,Call,11/19/24
1,85.0,62.42%,142.10,118.90,0.99442,243.68%,Call,11/15/24
2,90.0,60.21%,137.10,134.65,0.99425,229.34%,Call,12/6/24
3,95.0,58.00%,132.10,131.45,0.99853,178.30%,Call,10:48 ET
4,100.0,55.79%,127.13,129.89,0.99262,209.72%,Call,10:09 ET
5,105.0,53.58%,122.10,107.45,0.99526,182.71%,Call,12/2/24
6,110.0,51.37%,117.15,118.75,0.99212,185.63%,Call,09:37 ET
7,115.0,49.16%,112.13,104.39,0.99345,168.17%,Call,12/5/24
8,120.0,46.95%,107.28,105.10,0.99853,128.63%,Call,12/6/24
9,125.0,44.74%,102.25,101.82,0.98965,158.41%,Call,12/6/24


In [ ]:
df_calls = df[:59]

In [ ]:
df_calls

,Strike,Moneyness,Mid,Last,Delta,IV,Type,Time
0,80.0,64.63%,147.10,122.48,0.99459,258.94%,Call,11/19/24
1,85.0,62.42%,142.10,118.90,0.99442,243.68%,Call,11/15/24
2,90.0,60.21%,137.10,134.65,0.99425,229.34%,Call,12/6/24
3,95.0,58.00%,132.10,131.45,0.99853,178.30%,Call,10:48 ET
4,100.0,55.79%,127.13,129.89,0.99262,209.72%,Call,10:09 ET
5,105.0,53.58%,122.10,107.45,0.99526,182.71%,Call,12/2/24
6,110.0,51.37%,117.15,118.75,0.99212,185.63%,Call,09:37 ET
7,115.0,49.16%,112.13,104.39,0.99345,168.17%,Call,12/5/24
8,120.0,46.95%,107.28,105.10,0.99853,128.63%,Call,12/6/24
9,125.0,44.74%,102.25,101.82,0.98965,158.41%,Call,12/6/24


In [ ]:
df_puts = df[59:]

In [ ]:
df_puts

,Strike,Moneyness,Mid,Last,Delta,IV,Type,Time
59,80.0,-64.63%,0.01,0.01,-0.00026,181.28%,Put,11/27/24
60,85.0,-62.42%,0.01,0.01,-0.00027,171.12%,Put,11/26/24
61,90.0,-60.21%,0.01,0.01,-0.00029,161.54%,Put,11/12/24
62,95.0,-58.00%,0.01,0.01,-0.00030,152.47%,Put,11/26/24
63,100.0,-55.79%,0.01,0.01,-0.00032,143.88%,Put,12/6/24
64,105.0,-53.58%,0.01,0.01,-0.00034,135.70%,Put,12/4/24
65,110.0,-51.37%,0.01,0.01,-0.00036,127.89%,Put,12/6/24
66,115.0,-49.16%,0.01,0.01,-0.00038,120.43%,Put,12/6/24
67,120.0,-46.95%,0.01,0.01,-0.00074,119.69%,Put,09:57 ET
68,125.0,-44.74%,0.01,0.01,-0.00043,106.41%,Put,12/6/24


In [ ]:
calls = []
for i in range(len(df_calls)):
    temp = [df_calls.iloc[i]['Strike'], df_calls.iloc[i]['Mid']]
    calls.append(temp)
calls

[[80.0, 147.1],
 [85.0, 142.1],
 [90.0, 137.1],
 [95.0, 132.1],
 [100.0, 127.13],
 [105.0, 122.1],
 [110.0, 117.15],
 [115.0, 112.13],
 [120.0, 107.28],
 [125.0, 102.25],
 [130.0, 97.18],
 [135.0, 92.18],
 [140.0, 87.23],
 [145.0, 82.13],
 [150.0, 77.2],
 [155.0, 72.23],
 [160.0, 67.3],
 [165.0, 62.25],
 [170.0, 57.3],
 [175.0, 52.3],
 [177.5, 49.8],
 [180.0, 47.33],
 [182.5, 44.8],
 [185.0, 42.35],
 [187.5, 39.88],
 [190.0, 37.4],
 [192.5, 34.95],
 [195.0, 32.45],
 [197.5, 29.9],
 [200.0, 27.5],
 [202.5, 24.95],
 [205.0, 22.58],
 [207.5, 20.15],
 [210.0, 17.75],
 [212.5, 15.4],
 [215.0, 13.1],
 [217.5, 11.02],
 [220.0, 9.08],
 [222.5, 7.3],
 [225.0, 5.75],
 [227.5, 4.43],
 [230.0, 3.35],
 [232.5, 2.48],
 [235.0, 1.81],
 [237.5, 1.3],
 [240.0, 0.94],
 [242.5, 0.68],
 [245.0, 0.49],
 [247.5, 0.37],
 [250.0, 0.28],
 [255.0, 0.17],
 [260.0, 0.11],
 [265.0, 0.08],
 [270.0, 0.07],
 [275.0, 0.05],
 [280.0, 0.04],
 [285.0, 0.04],
 [290.0, 0.03],
 [300.0, 0.02]]

In [ ]:
puts = []
for i in range(len(df_puts)):
    temp = [df_puts.iloc[i]['Strike'], df_puts.iloc[i]['Mid']]
    puts.append(temp)
puts

[[80.0, 0.01],
 [85.0, 0.01],
 [90.0, 0.01],
 [95.0, 0.01],
 [100.0, 0.01],
 [105.0, 0.01],
 [110.0, 0.01],
 [115.0, 0.01],
 [120.0, 0.01],
 [125.0, 0.01],
 [130.0, 0.01],
 [135.0, 0.01],
 [140.0, 0.01],
 [145.0, 0.02],
 [150.0, 0.02],
 [155.0, 0.02],
 [160.0, 0.03],
 [165.0, 0.03],
 [170.0, 0.04],
 [175.0, 0.05],
 [177.5, 0.06],
 [180.0, 0.07],
 [182.5, 0.08],
 [185.0, 0.08],
 [187.5, 0.1],
 [190.0, 0.11],
 [192.5, 0.13],
 [195.0, 0.15],
 [197.5, 0.17],
 [200.0, 0.2],
 [202.5, 0.22],
 [205.0, 0.27],
 [207.5, 0.33],
 [210.0, 0.42],
 [212.5, 0.57],
 [215.0, 0.8],
 [217.5, 1.15],
 [220.0, 1.67],
 [222.5, 2.39],
 [225.0, 3.33],
 [227.5, 4.53],
 [230.0, 5.95],
 [232.5, 7.63],
 [235.0, 9.48],
 [237.5, 11.55],
 [240.0, 13.68],
 [242.5, 15.98],
 [245.0, 18.35],
 [247.5, 20.7],
 [250.0, 23.13],
 [255.0, 28.13],
 [260.0, 33.13],
 [265.0, 38.05],
 [270.0, 43.08],
 [275.0, 48.08],
 [280.0, 53.13],
 [285.0, 58.08],
 [290.0, 63.08],
 [300.0, 73.08]]

In [ ]:
r=0.0457
t=0.03
s=227.03
sigma=0.10

In [ ]:
from math import log, sqrt
from scipy.stats import norm

def calculateforcalls(calls, r, t, s, sigma):
    norm_calls = []
    for i in range(len(calls)):
        k = calls[i][0]
        Cd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Cd2 = Cd1 - sigma * sqrt(t)
        norm_calls.append([norm.cdf(Cd1), norm.cdf(Cd2)])
    return norm_calls

def calculateforputs(puts, r, t, s, sigma):
    norm_puts = []
    for i in range(len(puts)):
        k = puts[i][0]
        Pd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Pd2 = Pd1 - sigma * sqrt(t)
        norm_puts.append([norm.cdf(-Pd1), norm.cdf(-Pd2)])
    return norm_puts

norm_calls = calculateforcalls(calls, r, t, s, sigma)
norm_puts = calculateforputs(puts, r, t, s, sigma)

In [ ]:
from math import log, sqrt
from scipy.stats import norm

puts = []
for i in range(len(df_puts)):
    temp = [df_puts.iloc[i]['Strike'], df_puts.iloc[i]['Mid']]
    puts.append(temp)
puts

calls = []
for i in range(len(df_calls)):
    temp = [df_calls.iloc[i]['Strike'], df_calls.iloc[i]['Mid']]
    calls.append(temp)
calls

def calculateforcalls(calls, r, t, s, sigma):
    norm_calls = []
    for i in range(len(calls)):
        k = float(calls[i][0])
        Cd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Cd2 = Cd1 - sigma * sqrt(t)
        norm_calls.append([norm.cdf(Cd1), norm.cdf(Cd2)])
    return norm_calls

def calculateforputs(puts, r, t, s, sigma):
    norm_puts = []
    for i in range(len(puts)):
        k = float(puts[i][0])
        Pd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Pd2 = Pd1 - sigma * sqrt(t)
        norm_puts.append([norm.cdf(-Pd1), norm.cdf(-Pd2)])
    return norm_puts

r = 0.05
t = 1
s = 100
sigma = 0.2
norm_calls = calculateforcalls(calls, r, t, s, sigma)
norm_puts = calculateforputs(puts, r, t, s, sigma)
print("Call Option Norms (d1, d2):", norm_calls)
print("Put Option Norms (d1, d2):", norm_puts)


Call Option Norms (d1, d2): [[0.9286374026649281, 0.8971929255973333], [0.8775029982658564, 0.8321245076573099], [0.8097030607754923, 0.7507343889650786], [0.7278974800590385, 0.6578000563652848], [0.6368306511756191, 0.5596176923702425], [0.5422283335848053, 0.46257411156882894], [0.44964793063717595, 0.37200379301884023], [0.36361608589662114, 0.2915680223354189], [0.28719163790512714, 0.2231470638407554], [0.2219221296481797, 0.1670927159069422], [0.16806968271096834, 0.12265402553180338], [0.1249642711255824, 0.08842414661588854], [0.09137076551142581, 0.06271666534234163], [0.06580058486961333, 0.04383507925647721], [0.0467394204149981, 0.030236744884907966], [0.03279065998923093, 0.02061176742871457], [0.02274915223353259, 0.013902803796273823], [0.015624931538716434, 0.009289392788397906], [0.010635448380851674, 0.006154820184241009], [0.007181009288335104, 0.004047531015550542], [0.0058843974953245864, 0.0032739594742970542], [0.0048136866740260235, 0.0026440975438125377], [0.0

In [ ]:
norm_calls

[[0.9286374026649281, 0.8971929255973333],
 [0.8775029982658564, 0.8321245076573099],
 [0.8097030607754923, 0.7507343889650786],
 [0.7278974800590385, 0.6578000563652848],
 [0.6368306511756191, 0.5596176923702425],
 [0.5422283335848053, 0.46257411156882894],
 [0.44964793063717595, 0.37200379301884023],
 [0.36361608589662114, 0.2915680223354189],
 [0.28719163790512714, 0.2231470638407554],
 [0.2219221296481797, 0.1670927159069422],
 [0.16806968271096834, 0.12265402553180338],
 [0.1249642711255824, 0.08842414661588854],
 [0.09137076551142581, 0.06271666534234163],
 [0.06580058486961333, 0.04383507925647721],
 [0.0467394204149981, 0.030236744884907966],
 [0.03279065998923093, 0.02061176742871457],
 [0.02274915223353259, 0.013902803796273823],
 [0.015624931538716434, 0.009289392788397906],
 [0.010635448380851674, 0.006154820184241009],
 [0.007181009288335104, 0.004047531015550542],
 [0.0058843974953245864, 0.0032739594742970542],
 [0.0048136866740260235, 0.0026440975438125377],
 [0.0039314

In [ ]:
norm_puts

[[0.07136259733507186, 0.10280707440266668],
 [0.12249700173414357, 0.16787549234269006],
 [0.1902969392245078, 0.24926561103492145],
 [0.2721025199409614, 0.3421999436347152],
 [0.3631693488243809, 0.4403823076297575],
 [0.4577716664151948, 0.5374258884311711],
 [0.550352069362824, 0.6279962069811598],
 [0.6363839141033789, 0.7084319776645811],
 [0.7128083620948729, 0.7768529361592447],
 [0.7780778703518203, 0.8329072840930578],
 [0.8319303172890317, 0.8773459744681966],
 [0.8750357288744176, 0.9115758533841114],
 [0.9086292344885742, 0.9372833346576583],
 [0.9341994151303866, 0.9561649207435228],
 [0.9532605795850019, 0.969763255115092],
 [0.9672093400107691, 0.9793882325712854],
 [0.9772508477664674, 0.9860971962037262],
 [0.9843750684612835, 0.9907106072116021],
 [0.9893645516191484, 0.993845179815759],
 [0.9928189907116649, 0.9959524689844494],
 [0.9941156025046755, 0.9967260405257029],
 [0.995186313325974, 0.9973559024561874],
 [0.9960685382380303, 0.9978677257482123],
 [0.996793

In [ ]:
from math import log, sqrt
from scipy.stats import norm

def calculateforcalls(calls, r, t, s, sigma):
    implied_probs_calls = []
    for i in range(len(calls)):
        k = calls[i][0]
        Cd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Cd2 = Cd1 - sigma * sqrt(t)
        implied_probs_calls.append([k, norm.cdf(Cd2)])
    return implied_probs_calls

def calculateforputs(puts, r, t, s, sigma):
    implied_probs_puts = []
    for i in range(len(puts)):
        k = puts[i][0]
        Pd1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
        Pd2 = Pd1 - sigma * sqrt(t)
        implied_probs_puts.append([k, norm.cdf(-Pd2)])
    return implied_probs_puts

implied_probs_calls = calculateforcalls(calls, r, t, s, sigma)
implied_probs_puts = calculateforputs(puts, r, t, s, sigma)

print("Call Option Implied Probabilities:")
for strike, prob in implied_probs_calls:
    print(f"Strike: {strike}, Implied Probability: {prob:.4f}")

print("\nPut Option Implied Probabilities:")
for strike, prob in implied_probs_puts:
    print(f"Strike: {strike}, Implied Probability: {prob:.4f}")


Call Option Implied Probabilities:
Strike: 80.0, Implied Probability: 0.8972
Strike: 85.0, Implied Probability: 0.8321
Strike: 90.0, Implied Probability: 0.7507
Strike: 95.0, Implied Probability: 0.6578
Strike: 100.0, Implied Probability: 0.5596
Strike: 105.0, Implied Probability: 0.4626
Strike: 110.0, Implied Probability: 0.3720
Strike: 115.0, Implied Probability: 0.2916
Strike: 120.0, Implied Probability: 0.2231
Strike: 125.0, Implied Probability: 0.1671
Strike: 130.0, Implied Probability: 0.1227
Strike: 135.0, Implied Probability: 0.0884
Strike: 140.0, Implied Probability: 0.0627
Strike: 145.0, Implied Probability: 0.0438
Strike: 150.0, Implied Probability: 0.0302
Strike: 155.0, Implied Probability: 0.0206
Strike: 160.0, Implied Probability: 0.0139
Strike: 165.0, Implied Probability: 0.0093
Strike: 170.0, Implied Probability: 0.0062
Strike: 175.0, Implied Probability: 0.0040
Strike: 177.5, Implied Probability: 0.0033
Strike: 180.0, Implied Probability: 0.0026
Strike: 182.5, Implied 

In [ ]:
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_price(s, k, t, r, sigma, option_type):
    d1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    if option_type == 'call':
        price = s * norm.cdf(d1) - k * exp(-r * t) * norm.cdf(d2)
    elif option_type == 'put':
        price = k * exp(-r * t) * norm.cdf(-d2) - s * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price

def implied_volatility(option_price, s, k, t, r, option_type):
    def price_difference(sigma):
        return black_scholes_price(s, k, t, r, sigma, option_type) - option_price
    try:
        iv = brentq(price_difference, 1e-6, 5.0)
    except ValueError:
        iv = None
    return iv

call_vols = []
for strike, price in calls:
    iv = implied_volatility(price, s, strike, t, r, 'call')
    call_vols.append([strike, iv])

put_vols = []
for strike, price in puts:
    iv = implied_volatility(price, s, strike, t, r, 'put')
    put_vols.append([strike, iv])

print("Call Option Implied Volatilities (Strike, IV):")
for strike, iv in call_vols:
    print(f"Strike: {strike}, Implied Volatility: {iv:.4%}" if iv else f"Strike: {strike}, No Solution Found")

print("\nPut Option Implied Volatilities (Strike, IV):")
for strike, iv in put_vols:
    print(f"Strike: {strike}, Implied Volatility: {iv:.4%}" if iv else f"Strike: {strike}, No Solution Found")


Call Option Implied Volatilities (Strike, IV):
Strike: 80.0, No Solution Found
Strike: 85.0, No Solution Found
Strike: 90.0, No Solution Found
Strike: 95.0, No Solution Found
Strike: 100.0, No Solution Found
Strike: 105.0, No Solution Found
Strike: 110.0, No Solution Found
Strike: 115.0, No Solution Found
Strike: 120.0, No Solution Found
Strike: 125.0, No Solution Found
Strike: 130.0, Implied Volatility: 447.1096%
Strike: 135.0, Implied Volatility: 363.5203%
Strike: 140.0, Implied Volatility: 318.7773%
Strike: 145.0, Implied Volatility: 285.9573%
Strike: 150.0, Implied Volatility: 260.9597%
Strike: 155.0, Implied Volatility: 239.8355%
Strike: 160.0, Implied Volatility: 221.6607%
Strike: 165.0, Implied Volatility: 205.0582%
Strike: 170.0, Implied Volatility: 190.3323%
Strike: 175.0, Implied Volatility: 176.6031%
Strike: 177.5, Implied Volatility: 170.0917%
Strike: 180.0, Implied Volatility: 163.8630%
Strike: 182.5, Implied Volatility: 157.6378%
Strike: 185.0, Implied Volatility: 151.783

In [ ]:
call_vols = []
for strike, price in calls:
    iv = implied_volatility(price, s, strike, t, r, 'call')
    call_vols.append([iv])

put_vols = []
for strike, price in puts:
    iv = implied_volatility(price, s, strike, t, r, 'put')
    put_vols.append([iv])

print("Call Option Implied Volatilities (Strike, IV):")
for strike, iv in call_vols:
    print(f"Implied Volatility: {iv:.4%}" if iv else 'No Solution Found')

print("\nPut Option Implied Volatilities (Strike, IV):")
for strike, iv in put_vols:
    print(f"Implied Volatility: {iv:.4%}" if iv else 'No Solution Found')


In [ ]:
import matplotlib.pyplot as plt
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq
def black_scholes_price(s, k, t, r, sigma, option_type):
    d1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    if option_type == 'call':
        price = s * norm.cdf(d1) - k * exp(-r * t) * norm.cdf(d2)
    elif option_type == 'put':
        price = k * exp(-r * t) * norm.cdf(-d2) - s * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price
def implied_volatility(option_price, s, k, t, r, option_type):
    def price_difference(sigma):
        return black_scholes_price(s, k, t, r, sigma, option_type) - option_price
    try:
        iv = brentq(price_difference, 1e-6, 5.0)
    except ValueError:
        iv = None
    return iv
calls = [[95, 15], [100, 12], [105, 9], [110, 7], [115, 5]]
puts = [[95, 10], [100, 8], [105, 6], [110, 4.5], [115, 3]]
r = 0.05
t = 30 / 365
s = 105
call_vols = []
strikes = []
for strike, price in calls:
    iv = implied_volatility(price, s, strike, t, r, 'call')
    if iv is not None:
        call_vols.append(iv)
        strikes.append(strike)
plt.figure(figsize=(8, 6))
plt.plot(strikes, call_vols, marker='o', label='Call Implied Volatility')
plt.title("Volatility Smile for Call Options")
plt.xlabel("Strike Price")
plt.ylabel("Implied Volatility")
plt.grid(True)
plt.legend()
plt.show()
print("Call Option Implied Volatilities (Strike, IV):")
for i, iv in enumerate(call_vols):
    print(f"Strike: {strikes[i]}, Implied Volatility: {iv:.4%}")


In [ ]:
import matplotlib.pyplot as plt
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_price(s, k, t, r, sigma, option_type):
    d1 = (log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    if option_type == 'call':
        price = s * norm.cdf(d1) - k * exp(-r * t) * norm.cdf(d2)
    elif option_type == 'put':
        price = k * exp(-r * t) * norm.cdf(-d2) - s * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price

def implied_volatility(option_price, s, k, t, r, option_type):
    def price_difference(sigma):
        return black_scholes_price(s, k, t, r, sigma, option_type) - option_price
    try:
        iv = brentq(price_difference, 1e-6, 5.0)
    except ValueError:
        iv = None
    return iv

calls = [[95, 15], [100, 12], [105, 9], [110, 7], [115, 5]]
puts = [[95, 10], [100, 8], [105, 6], [110, 4.5], [115, 3]]
r = 0.05
t = 30 / 365
s = 105

call_vols = []
call_strikes = []
for strike, price in calls:
    iv = implied_volatility(price, s, strike, t, r, 'call')
    if iv is not None:
        call_vols.append(iv)
        call_strikes.append(strike)
put_vols = []
put_strikes = []
for strike, price in puts:
    iv = implied_volatility(price, s, strike, t, r, 'put')
    if iv is not None:
        put_vols.append(iv)
        put_strikes.append(strike)
plt.figure(figsize=(10, 6))
plt.plot(call_strikes, call_vols, marker='o', label='Call Implied Volatility', color='blue')
plt.plot(put_strikes, put_vols, marker='x', label='Put Implied Volatility', color='red')
plt.title("Volatility Smile for Call and Put Options")
plt.xlabel("Strike Price")
plt.ylabel("Implied Volatility")
plt.grid(True)
plt.legend()
plt.show()

# Output implied volatilities
print("Call Option Implied Volatilities (Strike, IV):")
for i, iv in enumerate(call_vols):
    print(f"Strike: {call_strikes[i]}, Implied Volatility: {iv:.4%}")

print("\nPut Option Implied Volatilities (Strike, IV):")
for i, iv in enumerate(put_vols):
    print(f"Strike: {put_strikes[i]}, Implied Volatility: {iv:.4%}")
